In [1]:
import numpy as np 
import pandas as pd 

In [3]:
df = pd.read_csv('../transactions_train.csv')
customers = pd.read_csv("../customers.csv")
articles = pd.read_csv("../articles.csv")

In [4]:
all_customers = df['customer_id'].unique().tolist()

In [89]:
df['t_dat']

0           2018-09-20
1           2018-09-20
2           2018-09-20
3           2018-09-20
4           2018-09-20
               ...    
31788319    2020-09-22
31788320    2020-09-22
31788321    2020-09-22
31788322    2020-09-22
31788323    2020-09-22
Name: t_dat, Length: 31788324, dtype: object

In [5]:
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)==True]

In [6]:
np.random.seed(44)
sampled_customers = np.random.choice(all_customers, size=int(1e5), replace=False, )
print(len(sampled_customers))
print(len(set(sampled_customers)))

100000
100000


In [7]:
df_sampled = df.loc[df['customer_id'].isin(sampled_customers)]
print(len(df_sampled.customer_id.unique()))

100000


In [8]:
len(df_sampled.customer_id.unique())

100000

In [9]:
print(df_sampled.shape)

(2325536, 5)


In [10]:
def query_based_on_date(df, start_date='2020-06-22', end_date='2020-09-22'):
    x = df[(df['t_dat'] > start_date) & (df['t_dat'] <= end_date)]
    return x

In [11]:
start_date = '2020-01-01'
x = query_based_on_date(df_sampled, start_date)
train_x = query_based_on_date(x, start_date, '2020-07-22')
test_x = query_based_on_date(x, '2020-07-22')


In [21]:
print(train_x.shape, test_x.shape)
sample_train_x = train_x.sample(n=10000)
print(sample_train_x.shape, test_x.shape)

(624354, 5) (89128, 5)
(10000, 5) (89128, 5)


In [29]:
train_customers = set(sample_train_x['customer_id'].unique().tolist())
test_customers = set(test_x['customer_id'].unique().tolist())
print(len(train_customers), len(test_customers))
print(type(train_customers), type(test_customers))

8267 19239
<class 'set'> <class 'set'>


In [30]:
train_test_overlap = train_customers.intersection(test_customers)
print(len(train_test_overlap) / len(test_customers))

0.18218202609283227


In [31]:
customers_to_remove = test_customers - train_test_overlap 
print(len(customers_to_remove))
test_customers-=customers_to_remove
print(len(test_customers))

15734
3505


In [35]:
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)==True]

test_x = filter_rows_by_values(test_x, 'customer_id', train_test_overlap)
print(test_x.shape)

(5510, 5)


In [33]:
# test_x_temp = test_x.sample(frac = 0.5)
# # # Creating dataframe with
# # # rest of the 50% values
# val_x = test_x.drop(test_x_temp.index)
# test_x = test_x_temp


In [36]:
print(sample_train_x.shape, test_x.shape)

(10000, 5) (5510, 5)


In [37]:
### training dataset : train_x
### testing dataset : test_x
### validation dataset : val_x

## Matrix Factorization

In [39]:
new_train_x = sample_train_x
new_test_x = test_x
new_val_x = val_x
print(new_train_x.shape, new_test_x.shape)
print(new_train_x.columns)
print(articles.columns)
print(customers.columns)

(10000, 5) (5510, 5)
Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'], dtype='object')
Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')
Index(['customer_id', 'FN', 'Active', 'club_member_status',
       'fashion_news_frequency', 'age', 'postal_code'],
      dtype='object')


In [55]:
new_train_x['Bought'] = 1
train_x['Bought'] = 1

print(new_train_x.columns, new_train_x.shape)



Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'Bought'],
      dtype='object') (10000, 6)


/var/folders/9l/1gmwtdm106j9pg3gthfjksxc0000gn/T/ipykernel_44598/3743888046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['Bought'] = 1


In [41]:
def reverse_lookup_iloc(test_index):
    return new_train_x.iloc[new_train_x.customer_id[test_customers_sample[test_index]]]

def reverse_lookup_index(test_index):
    return new_train_x.index[new_train_x.customer_id[test_customers_sample[test_index]]]


In [42]:
pivot_table = new_train_x.pivot_table(index='customer_id', columns='article_id', values='Bought').fillna(0)



In [50]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate


In [47]:
sample_train_x.iloc[0]

t_dat                                                      2020-05-30
customer_id         959fe4da631b3926f4a26562c90fb715fc3e5c834d61c0...
article_id                                                  817472002
price                                                        0.016932
sales_channel_id                                                    1
Bought                                                              1
Name: 26580260, dtype: object

In [52]:
sample_train_x['Bought'].max()

1

In [57]:
algo = SVD()

reader = Reader(rating_scale=(1, 2))

data = Dataset.load_from_df(train_x[['article_id', 'customer_id', 'Bought']], reader)
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0545  0.0544  0.0543  0.0542  0.0544  0.0544  0.0001  
MAE (testset)     0.0296  0.0293  0.0294  0.0293  0.0294  0.0294  0.0001  
Fit time          21.45   21.46   21.92   21.34   22.40   21.72   0.40    
Test time         0.67    0.69    0.67    0.48    0.79    0.66    0.10    


{'test_rmse': array([0.05452533, 0.05437706, 0.05429409, 0.05424586, 0.05441422]),
 'test_mae': array([0.02960936, 0.02933577, 0.0294213 , 0.02933106, 0.02936083]),
 'fit_time': (21.451756954193115,
  21.4644558429718,
  21.92227005958557,
  21.34296226501465,
  22.40249991416931),
 'test_time': (0.670935869216919,
  0.6855180263519287,
  0.6737310886383057,
  0.48476505279541016,
  0.7932169437408447)}

In [83]:
reader = Reader(rating_scale=(0, 1))
test_x['Bought'] = 1
train_data = Dataset.load_from_df(train_x[['customer_id', 'article_id', 'Bought']], reader)
test_data = Dataset.load_from_df(test_x[['customer_id', 'article_id', 'Bought']], reader)

trainset = train_data.build_full_trainset()

algo = SVD()

algo.fit(trainset)


/var/folders/9l/1gmwtdm106j9pg3gthfjksxc0000gn/T/ipykernel_44598/3431969800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['Bought'] = 1


In [84]:
testset = test_data.build_full_trainset().build_testset()

predictions = algo.test(testset)


In [85]:
len(test_x)

5510

In [86]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [87]:
top_n = get_top_n(predictions)

In [88]:
top_n

defaultdict(list,
            {'fb4c72e5597e6b64e8f14c4ae181346499bc1b75f1ba6e4544a31ad3bc8a5754': [(854602001,
               1),
              (801673014, 1),
              (867357001, 1),
              (915529003, 1),
              (914441004, 1),
              (867141003, 1),
              (754357005, 1),
              (831684005, 0.9567334311976233)],
             '6188584629b8cdfe870cf9d5c3496312e688463212120c1d3122765c850bfeed': [(884319008,
               1),
              (938182001, 1),
              (821336001, 1),
              (814655001, 0.9759713976186322),
              (608007001, 0.9659180792623862)],
             'fae179e3830e9773b52b6096951874986feec42f70b1bf936b98bc75528c0b77': [(906352001,
               1)],
             'ab2161020645d4c9fff9468701f82c9cce537b33663d5a5aac277fe6f3c3c6fd': [(854663002,
               1),
              (733097008, 0.974867333009891)],
             'd1fb7998088687412fde5aadd6f946c8fa1171cba0bd621e07aa85b10c1e8188': [(719655001,
     

In [ ]:
accuracy.rmse(predictions, verbose=True)

In [ ]:
all_ids = pivot_table.index.values.tolist()

R = pivot_table.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 10)

sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)


In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_table.columns)

In [ ]:
preds_df.shape

In [ ]:
preds_df.head(5)

In [ ]:
test_customers_sample = test_x.customer_id[0:5]
print(test_customers_sample)

In [ ]:
test_customers_sample_y = []
for ttt in test_customers_sample.items():
    train_indices = reverse_lookup_index(ttt[0])
    test_customers_sample_y.append(new_train_x.loc[train_indices]['article_id'].tolist())
    print(len(test_customers_sample_y[-1]))


In [ ]:
test_customer_predictions = preds_df

In [ ]:
preds_df.shape

In [ ]:
test_customers_sample_ycap = []
for ttt in test_customers_sample.items():
    train_indices = reverse_lookup_index(ttt[0])
    print(train_indices)
    preds_df.iloc[train_indices[0]].shape
    
    # test_customers_sample_y.append(new_train_x.loc[train_indices]['article_id'].tolist())
    # print(len(test_customers_sample_y[-1]))

In [ ]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # remove this case in advance
    # if not actual:
    #     return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])